In [50]:
# pip install langchain --upgrade
# Version: 0.0.164

# !pip install pypdf

In [51]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

Load you data

In [52]:
loader = PyPDFLoader("../content/nutrition/Manual-nutricion-dietetica-CARBAJAL.pdf")

In [53]:
data = loader.load()

In [54]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 367 document(s) in your data
There are 3330 characters in your document


Chunk your data up into smaller documents

In [55]:

# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [56]:
print (f'Now you have {len(texts)} documents')

Now you have 566 documents


Create embeddings of your documents to get ready for semantic search

In [57]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [58]:
# Check to see if there is an environment variable with you API keys, if not, use what you put below
OPENAI_API_KEY = ""

# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'YourAPIKey')
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-east1-gcp') # You may need to switch with your env

In [59]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [60]:
 # initialize pinecone
pinecone.init(      
	api_key='PINECONE_API_KEY',      
	environment='us-west1-gcp-free'      
)      
index = pinecone.Index('name_index') # put in the name of your pinecone index hereindex
index_name = 'name_index'

In [61]:
# Clear info index
# index.delete(delete_all=True)

In [62]:
texts[0]

Document(page_content='Manual\xa0de\xa0Nutrición \xa0y\xa0Dietética\xa0\nÁngeles\xa0Carbajal\xa0Azcona.\xa0Departamento \xa0de\xa0Nutrición. \xa0Facultad\xa0de\xa0Farmacia. \xa0Universidad \xa0Complutense \xa0de\xa0Madrid\xa0\nhttps://www.ucm.es/nutricioncarbajal/ \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa01\xa0\n\xa0\n\xa0\xa0\nManual\xa0de\xa0Nutrición \xa0y\xa0Dietética\xa0\n\xa0\n\xa0\xa0\xa0\xa0\n\xa0\n\xa0\n\xa0\n\xa0\xa0\xa0\xa0\nÁngeles\xa0Carbajal\xa0Azcona\xa0\nDepartamento \xa0de\xa0Nutric

In [63]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [64]:

query = "What is the Central Limit Theorem?"
docs = docsearch.similarity_search(query)

In [65]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

Manual de Nutrición  y Dietética 
Ángeles Carbajal Azcona. Departamento  de Nutrición.  Facultad de Farmacia.  Universidad  Complutense  de Madrid 
https://www.ucm.es/nutricioncarbajal/                                                                                                                                         13 
 
 en las zonas centrales del cuerpo. La relación circunferencia  de cintura / circunferencia  de cadera (RCC) 
permite esti


Query those docs to get your answer back

In [66]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [67]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [68]:
query = "What means to have noise in the data?"
# Noise in Data 93 / 188
docs = docsearch.similarity_search(query)

In [72]:
query = "Que son las proteinas?"
# Noise in Data 93 / 188
docs = docsearch.similarity_search(query)

In [73]:
docs

[Document(page_content='Manual\xa0de\xa0Nutrición \xa0y\xa0Dietética\xa0\nÁngeles\xa0Carbajal\xa0Azcona.\xa0Departamento \xa0de\xa0Nutrición. \xa0Facultad\xa0de\xa0Farmacia. \xa0Universidad \xa0Complutense \xa0de\xa0Madrid\xa0\nhttps://www.ucm.es/nutricioncarbajal/ \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa045\xa0\n\xa0\n\xa05. Proteínas \xa0\n\xa0\n\uf0b7 Proteínas \xa0\n\uf0b7 Aminoácidos \xa0esenciales \xa0y\xa0no\xa0esenciales \xa0\n\uf0b7 Calidad\xa0de\xa0la\xa0proteína\xa0\n\xa0\xa0\n\uf0b7 Pro

In [74]:
chain.run(input_documents=docs, question=query)

' Las proteínas son polímeros de aminoácidos unidos por enlaces peptídicos. Todos los tejidos vivos contienen proteínas. Son necesarias para el crecimiento, la reparación y la continua renovación de los tejidos corporales. Proporcionan energía (4 kcal/gramo) pero es poco recomendable utilizarlas para este fin.'